In [9]:
# ==========================================================
# 1. INSTALL LIBRARY
# ==========================================================
!pip install ultralytics gradio opencv-python-headless requests

import gradio as gr
import cv2
import requests
import os
from collections import deque
from ultralytics import YOLO
import torch


In [11]:

# ==========================================================
# 2. DOWNLOAD FILES (Model & Samples)
# ==========================================================
file_urls = {
    'best.pt': 'https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection/resolve/main/best.pt',
    'image_0.jpg': 'https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection/resolve/main/samples/riped_tomato_93.jpeg?download=true',
    'image_1.jpg': 'https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection/resolve/main/samples/unriped_tomato_18.jpeg?download=true',
    'video.mp4': 'https://huggingface.co/spaces/iamsuman/ripe-and-unripe-tomatoes-detection/resolve/main/samples/tomatoes.mp4?download=true'
}

def download_files():
    for name, url in file_urls.items():
        if not os.path.exists(name):
            print(f"📥 Downloading {name}...")
            r = requests.get(url, stream=True)
            with open(name, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    print("✅ Semua file siap!")

download_files()

# ==========================================================
# 3. LOAD MODEL
# ==========================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = YOLO('best.pt').to(device)

# ==========================================================
# 4. FUNGSI DETEKSI GAMBAR
# ==========================================================
def show_preds_image(image_path):
    if image_path is None: return None

    image = cv2.imread(image_path)
    results = model.predict(source=image_path, conf=0.25)
    result = results[0]
    names = model.names

    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        class_name = names[cls]

        # Warna: Merah untuk Ripe, Hijau untuk Unripe (Format BGR)
        color = (0, 0, 255) if 'unripe' not in class_name.lower() else (0, 255, 0)

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        label = f"{class_name}: {conf:.2f}"
        cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# ==========================================================
# 5. FUNGSI DETEKSI VIDEO (BATCH PROCESSING)
# ==========================================================
def show_preds_video_batch_centered(video_path, batch_size=8):
    if video_path is None: return None

    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    names = model.names

    total_ripe, total_unripe = set(), set()
    frame_buffer = []

    # Stream menggunakan YOLO track agar lebih akurat menghitung total objek unik
    results = model.track(source=video_path, persist=True, stream=True, conf=0.25, tracker="bytetrack.yaml")

    for r in results:
        frame = r.orig_img
        current_ripe_count = 0
        current_unripe_count = 0

        if r.boxes and r.boxes.id is not None:
            boxes = r.boxes.xyxy.cpu().numpy().astype(int)
            ids = r.boxes.id.cpu().numpy().astype(int)
            clss = r.boxes.cls.cpu().numpy().astype(int)

            for box, obj_id, cls in zip(boxes, ids, clss):
                x1, y1, x2, y2 = box
                name = names[cls]

                if 'unripe' not in name.lower():
                    color = (0, 0, 255)
                    total_ripe.add(obj_id)
                    current_ripe_count += 1
                else:
                    color = (0, 255, 0)
                    total_unripe.add(obj_id)
                    current_unripe_count += 1

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{name} ID:{obj_id}", (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # UI Overlay (Centered)
        current_text = f"Current → Ripe: {current_ripe_count} | Unripe: {current_unripe_count}"
        total_text = f"Total Seen → Ripe: {len(total_ripe)} | Unripe: {len(total_unripe)}"

        # Fungsi pembantu untuk center text
        def draw_centered_text(img, text, y_pos, color):
            (w, _), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
            cv2.putText(img, text, ((frame_width - w) // 2, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        draw_centered_text(frame, current_text, 40, (255, 255, 255))
        draw_centered_text(frame, total_text, 80, (0, 255, 255))

        yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    cap.release()

# ==========================================================
# 6. UI INTERFACE
# ==========================================================
image_path = [['image_0.jpg'], ['image_1.jpg']]
video_path = [['video.mp4']]

interface_image = gr.Interface(
    fn=show_preds_image,
    inputs=gr.Image(type="filepath", label="Input Image"),
    outputs=gr.Image(type="numpy", label="Result"),
    title="Tomato Ripeness Detection - Image",
    examples=image_path,
    cache_examples=False
)

interface_video = gr.Interface(
    fn=show_preds_video_batch_centered,
    inputs=gr.Video(label="Input Video"),
    outputs=gr.Image(type="numpy", label="Live Tracking"),
    title="Tomato Ripeness Detection - Video",
    examples=video_path,
    cache_examples=False
)

# Menyatukan ke dalam Tabbed Interface
demo = gr.TabbedInterface(
    [interface_image, interface_video],
    tab_names=['Image Inference', 'Video Inference'],
    title="🍅 Tomato Maturity Detection System"
)

if __name__ == "__main__":
    # share=True sangat penting untuk Google Colab
    demo.queue().launch(share=True, debug=True)

✅ Semua file siap!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d9f0e5f202ca665656.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 98ms
Prepared 1 package in 46ms
Installed 1 package in 0.97ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


video 1/1 (frame 1/155) /tmp/gradio/53b2b4f2c08f65439d3b72499de9e83f64724252948636f1d044f635a9bbcd4b/video.mp4: 640x384 7 unripes, 6 ripes, 37.8ms
video 1/1 (frame 2/155) /tmp/gradio/53b2b4f2c08f65439d3b72499de9e83f64724252948636f1d044f635a9bbcd4b/video.mp4: 640x384 7 unripes, 6 ripes, 8.6ms
video 1/1 (frame 3/155) /tmp/gradio/53b2b4f2c08f65439d3b72499de9e83f64724252948636f1d044f635a9bbcd4b/video.mp4: 640x384 7 unripes, 6 ripes, 8.9ms
video 1/1 (frame 4/155) /tmp/gradio/53b2b4f2c08f65439d3b72499de9e83f64724252948636f1d044f635a9bbcd4b/video.mp4: 640x384 7 unripes, 6 ripes, 9.0ms
video 1/1 (frame 5/155) /tmp/gradio/53b2b4f2c08f65439d3